In [ ]:
!wget https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/TP_PS/m2_ml2_tp_perceptron_structure.zip
!unzip m2_ml2_tp_perceptron_structure.zip

In [ ]:
DATA_PATH = 'data'

import pos_corpus as pcc
import id_feature as idfc

import discriminative_sequence_classifier as dsc
import numpy as np


Sujet TP2

# Perceptron structuré pour du POS tagging

Le POS tagging est une tâche de classification dite structurée. Il s'agit de prédire les tags syntaxiques (POS tags) des mots d'une phrase.  

# Chargement du dataset CoNLL

In [ ]:
corpus = pcc.PostagCorpus()
train_seq = corpus.read_sequence_list_conll(DATA_PATH + "/train-02-21.conll",
                                            max_sent_len=10, max_nr_sent=1000)

test_seq = corpus.read_sequence_list_conll(DATA_PATH + "/test-23.conll",
                                           max_sent_len=10, max_nr_sent=1000)

# nous n'utlisons pas le dev ici
# dev_seq = corpus.read_sequence_list_conll(DATA_PATH + "/dev-22.conll",
#                                           max_sent_len=10, max_nr_sent=1000)


Afficher des exemples de séquences.

In [ ]:
# on regarde des exemples de phrases
seq_ind = 3
seq = train_seq[seq_ind]

print(seq)
print('x', seq.x)
print('y', seq.y)


# Les *feature functions* ou *potentials* 

Nous allons dans un premier temps utiliser des feature functions qui mimiquent les HMM : 

*   *Initial features* : features en position 0 dans les phrases qui encodent l'identité des tags en position 0, exemple : init_tag:pron
*   *Emission features* : features mot/tag, appelés "nodes" (émission), exemple : id:many::adj
*   *Transition features* : features tag/tag, appelés "edges" (transition), exemple : prev_tag:adv::num
*   *Final features* : features en position finale dans les phrases: l'identité des tags en position finale, exemple : final_prev_tag:num


Dans le fichier ```id_feature.py```, est définie une classe ```IDFeatures``` qui crée ces features à partir des séquences mot/tag du train : 

In [ ]:
## Instancier et créer les feature functions sur le train 
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()


Afficher tous les noms de feature functions et les compter.

In [ ]:
nb = 0
for el in feature_mapper.??.??:
    print(el)
    nb += 1

print(nb)


Afficher tous les noms de feature functions de la séquence d'indice ```seq_ind```.

In [ ]:
current_feature_list = feature_mapper.feature_list[??]

??



# Le perceptron structuré (Collins, 2002)

Le perceptron struturé est un classifieur de séquences à séquences, dit discriminant. 

Ce type d'approche modélise la probabilité conditionnelle d'une séquence de tags $\boldsymbol y$ étant donnée une séquence de mots $\boldsymbol x$, à l'aide de produits scalaires entre le vecteur de poids $\boldsymbol w$ (à apprendre) et les *feature functions*, dont des exemples ont été donnés ci-dessus :  

\begin{equation} 
P(\boldsymbol y | \boldsymbol x ; \boldsymbol w) = \displaystyle\frac{1}{Z(\boldsymbol w, \boldsymbol x)}\exp \Big( \boldsymbol w \cdot \boldsymbol f_{\text{init}}(\boldsymbol x, y_0)+\sum_{i=0}^{N-2}\boldsymbol w \cdot \boldsymbol f_{\text{trans}}(i, \boldsymbol x, y_i, y_{i+1}) +\boldsymbol w  \cdot \boldsymbol f_{\text{final}}(\boldsymbol x, y_{N-1}) + \sum_{i=0}^{N-1}\boldsymbol w \cdot \boldsymbol f_{\text{emission}}(i, \boldsymbol x, y_i)\Big) 
\end{equation} 



Nous vous fournissons un squelette du code du perceptron structuré dans une cellule ci-dessous, à compléter.


La classe ```StructuredPerceptron``` hérite de la classe ```DiscriminativeSequenceClassifier```, qui elle-même hérite de la classe ```SequenceClassifier```. 

Ces deux classes sont données dans les fichiers respectifs ```discriminative_sequence_classifier.py``` et ```sequence_classifier.py```. 

Prenez le temps de lire ce que contiennent ces fichiers.

Voici le diagramme UML qui résume ces dépendances.


<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/TP_PS/diagramme_UML_structured_perceptron.png"
     alt="Digramme UML"
     style="float: left; margin-right: 1px;"
     />

## L'algorithme d'apprentissage

Initialiser les poids du perceptron avec le vecteur nul : $\boldsymbol w = \boldsymbol 0$

Pour $i=1\ldots T$

*   Pour chaque exemple d'apprentissage ($\boldsymbol x, \boldsymbol y$)

    1.    Générer une séquence de prédictions : $\boldsymbol z = argmax_{\boldsymbol z} \boldsymbol w \cdot 
\boldsymbol f (\boldsymbol x, \boldsymbol y)$
                 
    2.    Pour chaque Si $\boldsymbol z \neq \boldsymbol y$, faire : 
                 
\begin{equation*}
                    \boldsymbol w \leftarrow \boldsymbol w + \boldsymbol f (\boldsymbol x, \boldsymbol y) - \boldsymbol f (\boldsymbol x, \boldsymbol z)
\end{equation*}



La fonction $\boldsymbol f$ correspond aux feature functions extraits pour les séquences $\boldsymbol x, \boldsymbol z$, et sont accessibles à l'aide de l'objet ```feature_mapper```, vu ci-dessus.


## Travail à faire


Vous devez coder la méthode ```perceptron_update()``` de la classe ```StructuredPerceptron``` de la cellule ci-dessous.


Cette fonction prend en entrée ***une séquence*** et effectue les lignes 1 et 2 de l'algorithme sur cette séquence. 

Elle retourne ```num_labels, num_mistakes``` qui sont respectivement le nombre d'éléments de la séquence à traiter et le nombre d'erreurs commises par le modèle sur la séquence.

Détaillons ces deux lignes : 

1.   Pour générer la séquence de prédictions $\boldsymbol z$, faire un décodage Viterbi sur la séquence.

2.   Le vecteur de poids $\boldsymbol w$ du perceptron correspond à ```self.parameters```.

La mise à jour du vecteur est faite en testant chaque élément de la séquence prédite $z_i$ avec $i=0\ldots L-1$, avec $L$ la longueur de la séquence. Les features étant tous binaires, si une prédiction est fausse pour une position $i$, alors il faut ajouter ou retrancher 1 aux quatre types de feature functions. 

Plus précisément : 

*   Pour la première position dans la séquence ($i=0$) :
    *    si $z_0 \neq y_0$, faire : 

    \begin{eqnarray}
      \boldsymbol w[\text{initial features}((\boldsymbol x, \boldsymbol y_0))] & \mathrel{+}=&  1 \\
      \boldsymbol w[\text{initial features}((\boldsymbol x, \boldsymbol z_0))] & \mathrel{-}=&  1
    \end{eqnarray}
    
* Puis pour $i=0\ldots L-1$ : 
    *  si $z_i \neq y_i$, modifier $ \boldsymbol w$ de la même façon mais en considérant les *emission features*  et les *transition features*. Attention, les *transition features* ne sont pertinents que pour $i>0$.
* Enfin pour la dernière position $i=L-1$, il faut aussi considérer les *final features*.


***Aide***

Pour récupérer les quatre types de feature functions, ```feature_mapper``` a les méthodes suivantes :  

*    ```get_initial_features(sequence, y)``` 
*    ```get_emission_features(sequence, i, y)``` 
*    ```get_transition_features(sequence, i, y, y_prev)``` 
*    ```get_final_features(sequence, y_prev)``` 

Avec $i$ la position dans une séquence, ```y``` le tag à la position ```i``` de la vérité terrian ou bien issu de la prédiction, et ```y_prev``` le tag à la position ```i-1```, lorsqu'elle existe.

In [ ]:
class StructuredPerceptron(dsc.DiscriminativeSequenceClassifier):
    """ Implements Structured Perceptron"""

    def __init__(self, observation_labels, state_labels, feature_mapper,
                 num_epochs=10, learning_rate=1.0, averaged=True):
      
        dsc.DiscriminativeSequenceClassifier.__init__(self, observation_labels, state_labels, feature_mapper)
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.averaged = averaged
        self.params_per_epoch = []

    def train_supervised(self, dataset):
        self.parameters = np.zeros(self.feature_mapper.get_num_features())
        num_examples = dataset.size()
        for epoch in range(self.num_epochs):
            num_labels_total = 0
            num_mistakes_total = 0
            for i in range(num_examples):
                sequence = dataset.seq_list[i]
                num_labels, num_mistakes = self.perceptron_update(sequence)
                num_labels_total += num_labels
                num_mistakes_total += num_mistakes
            self.params_per_epoch.append(self.parameters.copy())
            acc = 1.0 - num_mistakes_total / num_labels_total
            print("Epoch: %i Accuracy: %f" % (epoch, acc))
        self.trained = True

        if self.averaged:
            new_w = 0
            for old_w in self.params_per_epoch:
                new_w += old_w
            new_w /= len(self.params_per_epoch)
            self.parameters = new_w

    def perceptron_update(self, sequence):
    
        # ----------
        # Exercice 
        num_labels, num_mistakes = 0, 0
        
        ?? 

        return num_labels, num_mistakes
        #
        # ----------

    def save_model(self, dir):
        fn = open(dir + "parameters.txt", 'w')
        for p_id, p in enumerate(self.parameters):
            fn.write("%i\t%f\n" % (p_id, p))
        fn.close()

    def load_model(self, dir):
        fn = open(dir + "parameters.txt", 'r')
        for line in fn:
            toks = line.strip().split("\t")
            p_id = int(toks[0])
            p = float(toks[1])
            self.parameters[p_id] = p
        fn.close()


Instancier le perceptron.

In [ ]:
sp = StructuredPerceptron(corpus.word_dict,
                          corpus.tag_dict,
                          feature_mapper)

Réaliser un entraînement sur 10 epochs : 

In [ ]:
??

Réaliser un décodage Viterbi et une évaluation sur le corpus de test. Afficher l'accuracy.

In [ ]:
??

print("Décodage Viterbi subset Test, acc=%.2f%%"%(100.*acc))

Réaliser un décodage Posterior et une évaluation sur le corpus de test. Afficher l'accuracy. Est-ce meilleur que Viterbi ?

In [ ]:
??

print("Décodage Posterior subset Test, acc=%.2f%%"%(100.*acc))

Y-a-t-il du sur-apprentissage ? 

In [ ]:
??

# Jeu de *feature functions* étendu


Tester à nouveau le perceptron mais cette fois avec les feature functions étendues mises à disposition dans le fichier ```extended_feature.py```.


Quels types de feature functions sont ajoutées par rapport à la classe précédente ```IDFeatures``` ?

